In [1]:
import pickle
import numpy as np
import torch
from LogLikelihoodTorch import log_likelihood,Optimizationloop, Initialize
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = torch.from_numpy(X)
torch.set_default_dtype(torch.float64)


In [10]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 7
p = 90

pi,kappa,mu = Initialize(p=p,K=K)
#%%

learning_rate = 0.5
n_iters = 1600

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)

In [11]:
# Optimizing Distribution:
torch.autograd.set_detect_anomaly(False)
Pi_fit,Kappa_fit,mu_fit = Optimizationloop(X_tensor,[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -5529658.767288913
epoch 101; Log-Likelihood = -6234742.619723307
epoch 201; Log-Likelihood = -6306555.475645827
epoch 301; Log-Likelihood = -6384355.667122153
epoch 401; Log-Likelihood = -6439671.321391728
epoch 501; Log-Likelihood = -6490537.36682027
epoch 601; Log-Likelihood = -6503085.7259873375
epoch 701; Log-Likelihood = -6525042.9047674425
epoch 801; Log-Likelihood = -6566001.727980787
epoch 901; Log-Likelihood = -6604339.715778856
epoch 1001; Log-Likelihood = -6637540.51473487
epoch 1101; Log-Likelihood = -6669598.811793188
epoch 1201; Log-Likelihood = -6701229.081594544
epoch 1301; Log-Likelihood = -6732754.815260561
epoch 1401; Log-Likelihood = -6764367.9418323925
epoch 1501; Log-Likelihood = -6796207.497722273


In [12]:
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [23]:
# Comparing Likelihood of pytorch vs. EM
import pickle
from WatsonDistribution import WatsonDistribution
MWD = WatsonDistribution(90)

K = 7
with open('EM.pickle','rb') as f:
    Pis_EM,kappas_EM,mus_EM,states = pickle.load(f)

#log_Likelihood_EM = sum(np.log([sum([ Pis_EM[j]* MWD.pdf(x,mus_EM[:,j],kappas_EM[j]) for j in range(K)]) for x in X.T]))
log_Likelihood_EM =  log_likelihood(X_tensor,torch.from_numpy(Pis_EM),torch.from_numpy(kappas_EM),torch.from_numpy(mus_EM),p=90,K=7)
log_Likelihood_torch = log_likelihood(X_tensor,pi_est,kappa_est,mu_est,p=90,K=7)

print("Likelihood Camparison:")
print(f"EM-Algortihem LogLikelihood: L = {log_Likelihood_EM}")
print(f"Torch Optimization of LogLikelihood: L = {log_Likelihood_torch}")

# Likelihood Camparison:
# EM-Algortihem LogLikelihood: L = 5642240.768217143

Likelihood Camparison:
EM-Algortihem LogLikelihood: L = 6469453.464885733
Torch Optimization of LogLikelihood: L = 6823658.060499564


In [13]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {pi_est[model]} , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.059989932087794615 , kappa = 382.06894902476273


Model 1: Pi = 0.1823821296441872 , kappa = 75.6881537487419


Model 2: Pi = 0.21236378739956965 , kappa = 61.15552134171554


Model 3: Pi = 0.16765849768230678 , kappa = 68.09194309757332


Model 4: Pi = 0.1612481194704363 , kappa = 85.90328170306847


Model 5: Pi = 0.16485506593634566 , kappa = 73.12527064507502


Model 6: Pi = 0.051502467779359885 , kappa = 863.0760443337671
